kurze Projektbeschreibung


In [1]:
import cv2
from ultralytics import YOLO
from PIL import Image
import numpy as np
import os
import csv
import pandas as pd


In [4]:
# Load YOLO pose model
model_pose = YOLO('yolo11n-pose.pt')


# Define training exercise and corresponding input and output folders

#input_folder = "videos/single_pushup_videos/"
#output_folder = "keyjoints/pushups/"

#input_folder = "videos/single_squat_videos/"
#output_folder = "keyjoints/squats/"

input_folder = "videos/single_pullup_videos/"
output_folder = "keyjoints/pullups/"

os.makedirs(output_folder, exist_ok=True)


Aus dem input Ordner werden single videos geladen und alle keypoints pro video in einer csv gespeichert.

In [5]:
# Process first all videos in the folder
video_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.mp4', '.mov'))] # add [:3] if only first three videos

for video_file in video_files:
    output_csv_path = os.path.join(output_folder, os.path.splitext(video_file)[0] + ".csv")
    if os.path.exists(output_csv_path):
        print(f"⏭️ Skipping {video_file} (CSV already exists)")
        continue
    video_path = os.path.join(input_folder, video_file)
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    fps = cap.get(cv2.CAP_PROP_FPS)

    output_rows = []
    xy_headers = [f"kp_{i}_{coord}" for i in range(17) for coord in ("x", "y")]
    conf_headers = [f"kp_{i}_conf" for i in range(17)]
    columns = ["frame", "time_sec"] + xy_headers + conf_headers

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        timestamp = frame_idx / fps
        results = model_pose.predict(source=frame, save=False, conf=0.25, verbose=False)

        for result in results:
            keypoints_xy = [np.nan] * (17 * 2)
            confidences = [np.nan] * 17

            try:
                keypoints = result.keypoints.xy[0].cpu().numpy()
                confidences = result.keypoints.conf[0].cpu().numpy()

                flattened_xy = []
                for i in range(17):
                    if i >= len(confidences) or confidences[i] < 0.2:
                        x, y = np.nan, np.nan
                    else:
                        x, y = keypoints[i]
                    flattened_xy.extend([x, y])
            except:
                flattened_xy = [np.nan] * (17 * 2)

            output_rows.append([frame_idx, timestamp] + flattened_xy + confidences.tolist())

        frame_idx += 1

    cap.release()

    # Save to CSV
    df_out = pd.DataFrame(output_rows, columns=columns)
    keypoint_cols = [col for col in df_out.columns if col.startswith("kp_")]
    df_out[keypoint_cols] = df_out[keypoint_cols].replace(0.0, np.nan)

    output_csv_path = os.path.join(output_folder, os.path.splitext(video_file)[0] + ".csv")
    df_out.to_csv(output_csv_path, index=False, na_rep="NaN")
    print(f"✅ Saved: {output_csv_path}")